In [2]:
import numpy as np

In [12]:
print(words[:10])

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn']


In [26]:
# read in all the words
words = open('../data/names.txt', 'r').read().splitlines()

In [31]:
data = open('../data/names.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print ('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 228145 characters, 27 unique.


In [17]:
print(ix_to_char)

{0: 'h', 1: '\n', 2: 'm', 3: 'x', 4: 'n', 5: 'c', 6: 'p', 7: 'a', 8: 'o', 9: 'y', 10: 'w', 11: 's', 12: 'z', 13: 'l', 14: 'b', 15: 'g', 16: 't', 17: 'd', 18: 'e', 19: 'k', 20: 'u', 21: 'v', 22: 'r', 23: 'f', 24: 'q', 25: 'i', 26: 'j'}


In [16]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [15]:
print(type(data))

<class 'str'>


In [18]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

In [33]:
# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden - (100, 27)
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden (100, 100)
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output (27, 100)
bh = np.zeros((hidden_size, 1)) # hidden bias (100, 1)
by = np.zeros((vocab_size, 1)) # output bias (27, 1)

In [36]:
def lossFun(inputs, targets, hprev):
    """
    inputs, targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    # Create four dictionaries
    xs, hs, ys, ps = {}, {}, {}, {}
    # Copy the last hidden state into hprev
    hs[-1] = np.copy(hprev)
    loss = 0
    
    # forward pass (each time step)
    for t in xrange(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
        # At this time step, input an encoded char, as a (27, 5) one hot vector
        xs[t][inputs[t]] = 1
        # At this time step, follow the formulas to get the hidden state at this time step
        # (100,27) @ (27, 5) = (100, 5), (100, 100) @ (100, 5) = (100, 5) + (100, 1) -> (100, n) (column vector of 100 outputs
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state

        # At this time step, compute the output state
        # (27, 100) @ (100, 5) = (27, 5) + (27, 1) = (5, 27)
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars

        # Normalize our probabiltiies using softmax
        # ps = (5, 27)
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars

        # Take the loss of the correct output probabilities
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    
    # backward pass: compute gradients going backwards
    # dWxh (100, 27), dWhh (100, 100), dWhy = (27, 100)
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    # dbh (100, 1),  dby (27, 1)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    ### ???
    dhnext = np.zeros_like(hs[0])

    # len(inputs) = 5
    for t in reversed(range(len(inputs))):
        # Backpropogate through the softmax to the logits
        # (27, 5)
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here

        # (27, 5) @ (5, 100) = (27, 100)
        dWhy += np.dot(dy, hs[t].T)
        
        # (27, 5) -> This secrety should sum across the columns dim = 1 to produce (27, 1)
        dby += dy

        # Backpropogate into the previous layer (100, 27) @ (27, 5) = (100, 5)
        dh = np.dot(Why.T, dy) + dhnext # backprop into h

        # Backprop through tanh nonlinearity (element-wise forward -> element-wise backward)
        dhraw = (1 - hs[t] * hs[t]) * dh 

        # 
        dbh += dhraw

        # dWxh = (100, 5) @ (5, 27) = (100, 27)
        dWxh += np.dot(dhraw, xs[t].T)

        # dWhh = (100, 5) (5, 100) = (100, 100)
        dWhh += np.dot(dhraw, hs[t-1].T)
        # dWhh = (100, 5) @ (5, 27) 
        dhnext = np.dot(Whh.T, dhraw)
    # Clipping gradients
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam)
    
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]
